In [1]:
import transformers
import evaluate
from datasets import load_dataset
from transformers import GenerationConfig
print(transformers.__version__)

/home/arooba/ssd/Shahzaib/.trans_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.36.2


In [2]:
model_checkpoint = "t5-small"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`Helsinki-NLP/opus-mt-en-ro`](https://huggingface.co/Helsinki-NLP/opus-mt-en-ro) checkpoint. 

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`. We use the English/Romanian part of the WMT dataset here.

In [3]:
raw_datasets = load_dataset("opus_books", "en-fr")
metric = evaluate.load("sacrebleu")

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 127085
    })
})

In [5]:
raw_datasets = raw_datasets["train"].train_test_split(test_size=0.2)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 101668
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 25417
    })
})

To access an actual element, you need to select a split first, then give an index:

In [6]:
raw_datasets["train"][0]

{'id': '55848',
 'translation': {'en': 'Only joy made itself felt, only pride dominated.',
  'fr': 'Seulement la joie perçait, l’orgueil dominait.'}}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [7]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(raw_datasets["train"])

,id,translation
0,89505,"{'en': '""A person can be both a cannibal and a decent man,"" Conseil replied, ""just as a person can be both gluttonous and honorable.', 'fr': '-- On peut être anthropophage et brave homme, répondit Conseil, comme on peut être gourmand et honnête.'}"
1,46917,"{'en': 'It was not difficult to conquer, as she had hitherto done, men prompt to let themselves be seduced, and whom the gallant education of a court led quickly into her net. Milady was handsome enough not to find much resistance on the part of the flesh, and she was sufficiently skillful to prevail over all the obstacles of the mind.', 'fr': 'Il n'était pas difficile de vaincre, ainsi qu'elle l'avait fait jusque-là, des hommes prompts à se laisser séduire, et que l'éducation galante de la cour entraînait vite dans le piège; Milady était assez belle pour ne pas trouver de résistance de la part de la chair, et elle était assez adroite pour l'emporter sur tous les obstacles de l'esprit.'}"
2,103384,"{'en': 'He was still weak, and strict diet had been imposed upon him, but no access of fever supervened.', 'fr': 'Il était faible encore, et une diète sévère lui avait été imposée, mais aucun accès n'était revenu.'}"
3,108962,"{'en': 'I nodded my head up and down.', 'fr': 'Je remuai la tête de haut en bas.'}"
4,50413,"{'en': '""Pardon! I've there known poor mothers of families, virtuous women, I assure you, real saints, who wanted even bread.""', 'fr': '-- Pardonnez-moi! j’ai connu là de pauvres mères de famille, des femmes vertueuses, je vous assure, de véritables saintes, qui manquaient même de pain.'}"


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [9]:
metric

EvaluationModule(name: "sacrebleu", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'e

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings (list of list for the labels):

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, max_input_length=128, max_target_length=128)

For the mBART tokenizer (like we have here), we need to set the source and target languages (so the texts are preprocessed properly). You can check the language codes [here](https://huggingface.co/facebook/mbart-large-cc25) if you are using this notebook on a different pairs of languages.

In [11]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "fr-XX"

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

If you are using one of the five T5 checkpoints that require a special prefix to put before the inputs, you should adapt the following cell.

In [12]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to French: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [13]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "fr"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    labels = tokenizer(text_target=targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [14]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[13959, 1566, 12, 2379, 10, 3462, 3922, 263, 1402, 1800, 6, 163, 6064, 3, 16032, 5, 1], [13959, 1566, 12, 2379, 10, 96, 18741, 6, 30643, 4609, 1380, 3, 26, 22, 7754, 9, 11260, 29, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[679, 83, 1194, 50, 14684, 15, 399, 11666, 155, 6, 3, 40, 22, 127, 5398, 173, 10138, 9, 155, 5, 1], [1636, 262, 107, 923, 6, 30643, 58, 2173, 9, 3, 26, 31, 7754, 9, 11260, 29, 5, 1]]}

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [15]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map: 100%|██████████| 25417/25417 [00:01<00:00, 17141.98 examples/s]


## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [16]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [18]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [19]:
my_config = GenerationConfig.from_pretrained(model_checkpoint)

my_config.max_new_tokens = 128

In [20]:
batch_size = 128
model_name = model_checkpoint.split("/")[-1]
exp = "3"
args = Seq2SeqTrainingArguments(
    output_dir="trained/t5-small"+"_exp"+exp,
    evaluation_strategy = "steps",
    learning_rate=8e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    logging_dir="logs/t5-small"+"_exp"+exp,
    logging_strategy = "steps",
    logging_steps = 500,
    load_best_model_at_end = True,
    metric_for_best_model ="bleu",
    generation_config=my_config
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [21]:
trainer.model.generation_config

GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "max_new_tokens": 128,
  "pad_token_id": 0
}

We can now finetune our model by just calling the `train` method:

In [22]:
trainer.train()

  3%|▎         | 500/15900 [02:35<1:21:30,  3.15it/s]

{'loss': 1.9121, 'learning_rate': 0.007748427672955975, 'epoch': 0.63}


                                                     
  3%|▎         | 500/15900 [09:35<1:21:30,  3.15it/s]

{'eval_loss': 1.549000859260559, 'eval_bleu': 22.2316, 'eval_gen_len': 42.2604, 'eval_runtime': 419.087, 'eval_samples_per_second': 60.649, 'eval_steps_per_second': 0.475, 'epoch': 0.63}


  6%|▋         | 1000/15900 [12:15<1:09:39,  3.56it/s]  

...

                                                       
 97%|█████████▋| 15500/15900 [3:23:57<02:05,  3.18it/s]

{'eval_loss': 1.3010462522506714, 'eval_bleu': 30.1474, 'eval_gen_len': 41.3291, 'eval_runtime': 275.2978, 'eval_samples_per_second': 92.325, 'eval_steps_per_second': 0.723, 'epoch': 19.5}


100%|██████████| 15900/15900 [3:26:05<00:00,  1.29it/s]

{'train_runtime': 12365.4666, 'train_samples_per_second': 164.439, 'train_steps_per_second': 1.286, 'train_loss': 1.0802411003832548, 'epoch': 20.0}


TrainOutput(global_step=15900, training_loss=1.0802411003832548, metrics={'train_runtime': 12365.4666, 'train_samples_per_second': 164.439, 'train_steps_per_second': 1.286, 'train_loss': 1.0802411003832548, 'epoch': 20.0})

### Inference

In [ ]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."

In [ ]:
from transformers import pipeline

translator = pipeline("translation_en_to_fr", model="/home/arooba/ssd/Shahzaib/QuickFormer_Research_Project/machine translation/trained/t5-small/checkpoint-500/")
output = translator(text)

In [ ]:
preds = [item["translation_text"] for item in output]
preds

### Without Pipeline

In [ ]:
from transformers import AutoTokenizer
PATH = "/home/arooba/ssd/Shahzaib/QuickFormer_Research_Project/machine translation/trained/t5-small/checkpoint-500/"
tokenizer = AutoTokenizer.from_pretrained(PATH)
inputs = tokenizer(text, return_tensors="pt", padding=True).input_ids
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(PATH)
outputs = model.generate(inputs, max_new_tokens=120, do_sample=True, top_k=30, top_p=0.95)

In [ ]:
preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
preds

### BLEU Score

In [23]:
def evaluate_bleu_pipeline(dataset, translator):
    input = [trans["en"] for trans in dataset["translation"]]
    # text_list_en
    target = [trans["fr"] for trans in dataset["translation"]]
    output = translator(input)
    preds = [item["translation_text"] for item in output]
    bleu = evaluate.load("bleu")
    results = bleu.compute(predictions=preds, references=target)
    return results

def evaluate_bleu(dataset, model_path):
    input = [trans["en"] for trans in dataset["translation"]]
    # text_list_en
    target = [trans["fr"] for trans in dataset["translation"]]
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    inputs = tokenizer(input, return_tensors="pt", padding=True).input_ids
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    outputs = model.generate(inputs, max_new_tokens=120, do_sample=True, top_k=30, top_p=0.95)
    preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    bleu = evaluate.load("bleu")
    results = bleu.compute(predictions=preds, references=target)
    return results


In [26]:
# bleu = evaluate_bleu(raw_datasets["train"][:10], PATH)
from transformers import pipeline
translator = pipeline("translation_en_to_fr", model="/home/arooba/ssd/Shahzaib/QuickFormer_Research_Project/machine translation/trained/t5-small_exp3/checkpoint-13500/")
bleu = evaluate_bleu_pipeline(raw_datasets["test"][:500], translator)

In [27]:
print(bleu)

{'bleu': 0.30277998869958345, 'precisions': [0.6223143292167146, 0.3772301713478184, 0.2554272517321016, 0.178702807357212], 'brevity_penalty': 0.9410733209365135, 'length_ratio': 0.9427432216905901, 'translation_length': 11822, 'reference_length': 12540}
